# Chronicles of higher education job scraper

Collecting all job advertisements for tenure-track for North American four-year institutions.

- **[Query](https://jobs.chronicle.com/jobs/faculty-positions/north-america/tenured-tenured-track/)**


Everytime you scrape:

1. Load in previous job advertisements
2. Scrape all the *new job advertisements*
3. De-duplicate if necessary
4. Output to DB/CSV


In [ ]:
# Data manipulation libraries
import pandas as pd
import numpy as np

# Webscraping libaries and tools
import requests
from bs4 import BeautifulSoup as bs
import re
import time
from tqdm.notebook import tqdm
tqdm.pandas()

# reading path to data files
from glob import glob

In [ ]:
def parse_list_page_item(list_item):
    """
        Takes the list item HTML and parses out the four fields below into a list
    
    """
    title_tag = list_item.find("h3").find("a")
    job_title = title_tag.text
    job_url_suffix = title_tag['href'].strip()
    job_id = job_url_suffix.split("/")[2]
    job_url = "https://jobs.chronicle.com{}".format(job_url_suffix)
    diversity_job = False if list_item.find("p",attrs={"class":"ribbon"}) is None else True
    return [int(job_id),job_title,job_url,diversity_job]

def parse_list_page(url):
    """
        Returns the basic info from the jobs listing page
        
        || job id || job title || url || diversity job? 
    
    """
    time.sleep(1)
    r = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0'})
    # The part of the webpage with the id tag "listing" contains all the job postings
    listing_page = bs(r.text).find("ul",attrs={"id":'listing'})
    # Parse out the ads
    list_items = listing_page.findAll("li",attrs={"id": re.compile("item-[0-9]+")})
    parsed_list_page = [parse_list_page_item(li) for li in list_items]
    return pd.DataFrame(parsed_list_page,columns=["Job ID","Job Title","Job URL","Diversity Job"]).set_index("Job ID")



In [ ]:
# Get Job ID for most recent date posted which already exists

list_of_csv_files = glob("../data/*")
most_recent_csv = sorted(list_of_csv_files, reverse=True)[0]
ls_df = pd.read_csv(most_recent_csv).sort_values("Date Posted",ascending=False)
already_scraped = set(ls_df['Job ID'])
ls_df

In [ ]:
url = "https://jobs.chronicle.com/jobs/faculty-positions/north-america/tenured-tenured-track/{}"

frames = []
job_ids = set()
page = 1
new_jobs = True
while new_jobs:
    print(page,end=" ")
    frame = parse_list_page(url.format(page))
    prev_job_ids = job_ids
    job_ids = set(frame.index)
    new_jobs = not (bool(job_ids.intersection(already_scraped)) or (prev_job_ids == job_ids))
    if bool(job_ids.intersection(already_scraped)): print(job_ids.intersection(already_scraped))
    frames.append(frame)
    page +=1

listing_df = pd.concat(frames)
listing_df = listing_df[~listing_df.index.isin(already_scraped)]
listing_df

In [ ]:
def get_description_of_page(soup_page):
    """
        Parses the beautiful-soup object of the page response for the job description.
        
        :param soup_page: The beautiful soup object that contains the desired page.
        :returns: The text of the job description.
    """
    description = soup_page.find("div",attrs={"class":"mds-edited-text mds-font-body-copy-bulk"}).get_text()
    return description
    

In [ ]:
def get_details_block_of_page(soup_page):
    """
        Every page has a set of details that contains information like who the employer for a job is, location, etc.
        Parses the beautiful-soup object of the page for the summary of the details of the job.
        
        :param soup_page: The beautiful soup object that contains the desired page.
        :returns: The beautiful soup tag for the details. Gets parsed for the important details later.
    """
    details_block = soup_page.find_all("dl",attrs={"class":"mds-list mds-list--definition mds-list--border mds-margin-bottom-b0"})
    return details_block
    

In [ ]:
def link_keys_and_values(list_of_keys_and_values):
    """
        Takes a list of alternating elements with key and value class elements and pairs them up. The current version
        of the website has a lot of information stored in a weird format where one element has a class called 'mds-list__key'
        and the element below it contains a class called 'mds-list__value'. this function matches those two together.
        
        :param list_of_keys_and_values: List of soup elements that have alternating key and value class attributes.
        :returns: A dictionary where the key and value correspond to the keys and value in the html. The keys and values are 
        just the text from the element.
    """
    dictionary_form = {}
    key = None
    value = None
    for element in list_of_keys_and_values:
        if "mds-list__key" in element.get("class"):
            key = element
        if "mds-list__value" in element.get("class"):
            value = element
            if key != None and value != None:
                dictionary_form[key.get_text().strip()] = value.get_text().strip()
            key = None
            value = None  

    return dictionary_form

In [ ]:
def aggregate_children_of_elements(list_of_elements):
    """
        Takes a list of elements with children and gathers them together.
        
        :param list_of_elements: List of beautiful soup elements.
        :returns: A list of all the children of the elements in the input list.
    """
    children = []
    for element in list_of_elements:
        for child in element.findChildren(recursive=False):
            children.append(child)
    
    return children

In [ ]:
def parse_details_page(url):
    """
        Parses the details page of a university
        
        || employer || location || salary || posted date || position_type (list) || description
    
    """
    time.sleep(0.25)
    r = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0'})
    details_page = bs(r.text)
    
    description = get_description_of_page(details_page)
    
    details_block = get_details_block_of_page(details_page)
    list_of_keys_and_values = aggregate_children_of_elements(details_block)
    details_dict = link_keys_and_values(list_of_keys_and_values)
    
    employer,location,salary,posted_date,position_type = None,None,None,None,None
    
    employer = None if "Employer" not in details_dict else details_dict["Employer"]
    location = None if "Location" not in details_dict else details_dict["Location"]    
    salary = None if "Salary" not in details_dict else details_dict["Salary"]
    posted_date = None if "Posted Date" not in details_dict else details_dict["Posted Date"] # not sure if the "start date" is the Date Posted
    try:
        position_type = None if "Position Type" not in details_dict else details_dict["Position Type"]
        position_type = [text.strip() for text in position_type.split(",")]
    except:
        pass
    
    return employer,location,salary,posted_date,position_type,description


In [ ]:
listing_df

In [ ]:
listing_df[['Employer',
            'Location',
            'Salary',
            'Date Posted',
            'position_type',
            'Description']] = listing_df.progress_apply(lambda row: parse_details_page(row['Job URL']),
                                                        axis=1,
                                                        result_type='expand')
listing_df["Date Posted"] = pd.to_datetime(listing_df["Date Posted"],infer_datetime_format=True)

In [ ]:
listing_df = listing_df[listing_df['position_type'].notna()]

In [ ]:
position_type = pd.DataFrame(listing_df['position_type'].values.tolist(),
                             index=listing_df.index).fillna(np.nan)
position_type = position_type.rename(columns = lambda x: (x/10)).add_prefix('Position Type ')
print("{}x{}".format(*listing_df.shape))

merged_df = pd.merge(listing_df,
                     position_type,
                     how="left",
                     left_index=True,
                     right_index=True)
print("{}x{}".format(*merged_df.shape))
merged_df = merged_df.drop("position_type",axis=1)
print("{}x{}".format(*merged_df.shape))
merged_df = merged_df.sort_values("Date Posted",ascending=False)
merged_df

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d")
merged_df.to_csv(f"../data/{timestamp}-chronicles_of_higher_ed.csv")